In [1]:
#!/usr/bin/env python
# coding: utf-8

# # BinaryNet on CIFAR10
# 
# <a href="https://colab.research.google.com/github/larq/docs/blob/master/docs/larq/tutorials/binarynet_cifar10.ipynb"><button class="notebook-badge">Run on Colab</button></a> <a href="https://github.com/larq/docs/blob/master/docs/larq/tutorials/binarynet_cifar10.ipynb"><button class="notebook-badge">View on GitHub</button></a>
# 
# In this example we demonstrate how to use Larq to build and train BinaryNet on the CIFAR10 dataset to achieve a validation accuracy approximately 83% on laptop hardware.
# On a Nvidia GTX 1050 Ti Max-Q it takes approximately 200 minutes to train. For simplicity, compared to the original papers [BinaryConnect: Training Deep Neural Networks with binary weights during propagations](https://arxiv.org/abs/1511.00363), and [Binarized Neural Networks: Training Deep Neural Networks with Weights and Activations Constrained to +1 or -1](https://arxiv.org/abs/1602.02830), we do not impliment learning rate scaling, or image whitening.

# In[1]:




# In[2]:


import tensorflow as tf
import larq as lq
import numpy as np
import matplotlib.pyplot as plt
from keras_preprocessing.image import ImageDataGenerator

# ## Import CIFAR10 Dataset
# 
# We download and normalize the CIFAR10 dataset.

# In[3]:


num_classes = 10

(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.cifar10.load_data()

train_images = train_images.reshape((50000, 32, 32, 3)).astype("float32")
test_images = test_images.reshape((10000, 32, 32, 3)).astype("float32")

# Normalize pixel values to be between -1 and 1
train_images, test_images = train_images / 127.5 - 1, test_images / 127.5 - 1

val_images, val_labels = test_images[:1000], test_labels[:1000]
test_images, test_labels = test_images[1000:], test_labels[1000:]

# ## Build BinaryNet
# 
# Here we build the BinaryNet model layer by layer using the [Keras Sequential API](https://www.tensorflow.org/guide/keras).

# In[4]:


# All quantized layers except the first will use the same options
kwargs = dict(input_quantizer="ste_sign",
              kernel_quantizer="ste_tern",
              kernel_constraint="weight_clip",
              use_bias=False)

model = tf.keras.models.Sequential([
    # In the first layer we only quantize the weights and not the input
    lq.layers.QuantConv2D(128, 3, input_shape=(32, 32, 3), padding="same", **kwargs),
    tf.keras.layers.BatchNormalization(momentum=0.999, scale=False),

    lq.layers.QuantConv2D(128, 3, padding="same", **kwargs),
    tf.keras.layers.BatchNormalization(momentum=0.999, scale=False),
    tf.keras.layers.MaxPool2D(pool_size=(2, 2), strides=(2, 2)),

    lq.layers.QuantConv2D(256, 3, padding="same", **kwargs),
    tf.keras.layers.BatchNormalization(momentum=0.999, scale=False),

    lq.layers.QuantConv2D(256, 3, padding="same", **kwargs),
    tf.keras.layers.BatchNormalization(momentum=0.999, scale=False),
    tf.keras.layers.MaxPool2D(pool_size=(2, 2), strides=(2, 2)),

    lq.layers.QuantConv2D(512, 3, padding="same", **kwargs),
    tf.keras.layers.BatchNormalization(momentum=0.999, scale=False),

    lq.layers.QuantConv2D(512, 3, padding="same", **kwargs),
    tf.keras.layers.BatchNormalization(momentum=0.999, scale=False),
    tf.keras.layers.MaxPool2D(pool_size=(2, 2), strides=(2, 2)),
    tf.keras.layers.Flatten(),

    lq.layers.QuantDense(1024, **kwargs),
    tf.keras.layers.BatchNormalization(momentum=0.999, scale=False),
    tf.keras.layers.Dropout(0.5),

    lq.layers.QuantDense(1024, **kwargs),
    tf.keras.layers.BatchNormalization(momentum=0.999, scale=False),
    tf.keras.layers.Dropout(0.5),

    lq.layers.QuantDense(10, **kwargs),
    tf.keras.layers.BatchNormalization(momentum=0.999, scale=False),
    tf.keras.layers.Activation("softmax")
])



# One can output a summary of the model:

# In[5]:


lq.models.summary(model)


# ## Model Training
# 
# Compile the model and train the model

# In[6]:



model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

eval_only = True
if eval_only:
    model.load_weights('fullsame_best')
    print("******************** BEST *******************************")
    print("Training")
    model.evaluate(train_images, train_labels)
    model.evaluate(train_images[:100], train_labels[:100])
    print("Test")
    model.evaluate(test_images, test_labels)
    print("Val")
    model.evaluate(val_images, val_labels)

2023-03-25 10:03:24.039247: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-25 10:03:24.164232: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-03-25 10:03:24.164249: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-03-25 10:03:24.189555: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-03-25 10:03:24.796601: W tensorflow/stream_executor/platform/de

+sequential stats------------------------------------------------------------------------------+
| Layer                  Input prec.            Outputs  # 2-bit  # 32-bit  Memory  2-bit MACs |
|                              (bit)                         x 1       x 1    (kB)             |
+----------------------------------------------------------------------------------------------+
| quant_conv2d                     1   (-1, 32, 32, 64)     1728         0    0.42     1769472 |
| batch_normalization              -   (-1, 32, 32, 64)        0       128    0.50           0 |
| quant_conv2d_1                   1   (-1, 32, 32, 64)    36864         0    9.00    37748736 |
| batch_normalization_1            -   (-1, 32, 32, 64)        0       128    0.50           0 |
| max_pooling2d                    -   (-1, 16, 16, 64)        0         0       0           0 |
| quant_conv2d_2                   1  (-1, 16, 16, 128)    73728         0   18.00    18874368 |
| batch_normalization_2       

2023-03-25 10:03:27.844828: W tensorflow/core/util/tensor_slice_reader.cc:96] Could not open halfsame_best: FAILED_PRECONDITION: halfsame_best; Is a directory: perhaps your file is in a different file format and you need to use a different restore operator?
2023-03-25 10:03:28.219376: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 614400000 exceeds 10% of free system memory.


4/4 [==============================] - 0s 65ms/step - loss: 1.4662 - accuracy: 0.5200
Test
282/282 [==============================] - 24s 86ms/step - loss: 1.6126 - accuracy: 0.5437
Val
32/32 [==============================] - 3s 83ms/step - loss: 1.4764 - accuracy: 0.5790


In [2]:
from keras.models import Model
model.load_weights('fullsame_best')

layer_name = 'quant_conv2d'
outtermediate_layer_model = Model(inputs=model.input,outputs=model.get_layer(layer_name).output)
outtermediate_output = outtermediate_layer_model.predict(train_images[:1])
arr = np.loadtxt("conv1.txt", delimiter=",", dtype=int).reshape(outtermediate_output.shape)
diff = np.abs(arr- outtermediate_output)
print("Layer 1 Conv", np.sum(diff))

layer_name = 'batch_normalization'
outtermediate_layer_model = Model(inputs=model.input,outputs=model.get_layer(layer_name).output)
outtermediate_output = outtermediate_layer_model.predict(train_images[:1])
outtermediate_output[outtermediate_output>=0] = 1
outtermediate_output[outtermediate_output<0] = 0
arr = np.loadtxt("sign1.txt", delimiter=",", dtype=int).reshape(outtermediate_output.shape)
diff = np.abs(arr- outtermediate_output)
print("Layer 1 Sign", np.sum(diff))

layer_name = 'quant_conv2d_1'
outtermediate_layer_model = Model(inputs=model.input,outputs=model.get_layer(layer_name).output)
outtermediate_output = outtermediate_layer_model.predict(train_images[:1])
arr = np.loadtxt("conv2.txt", delimiter=",", dtype=int).reshape(outtermediate_output.shape)
diff = np.abs(arr- outtermediate_output)
print("Layer 2 Conv", np.sum(diff))

layer_name = 'batch_normalization_1'
outtermediate_layer_model = Model(inputs=model.input,outputs=model.get_layer(layer_name).output)
outtermediate_output = outtermediate_layer_model.predict(train_images[:1])
outtermediate_output[outtermediate_output>=0] = 1
outtermediate_output[outtermediate_output<0] = 0
arr = np.loadtxt("sign2.txt", delimiter=",", dtype=int).reshape(outtermediate_output.shape)
diff = np.abs(arr - outtermediate_output)
print("Layer 2 Sign", np.sum(diff))

layer_name = 'quant_conv2d_2'
outtermediate_layer_model = Model(inputs=model.input,outputs=model.get_layer(layer_name).output)
outtermediate_output = outtermediate_layer_model.predict(train_images[:1])
arr = np.loadtxt("conv3.txt", delimiter=",", dtype=int).reshape(outtermediate_output.shape)
diff = np.abs(arr- outtermediate_output)
print("Layer 3 Conv", np.sum(diff))

layer_name = 'quant_conv2d_3'
outtermediate_layer_model = Model(inputs=model.input,outputs=model.get_layer(layer_name).output)
outtermediate_output = outtermediate_layer_model.predict(train_images[:1])
arr = np.loadtxt("conv4.txt", delimiter=",", dtype=int).reshape(outtermediate_output.shape)
diff = np.abs(arr- outtermediate_output)
print("Layer 4 Conv", np.sum(diff))

layer_name = 'quant_conv2d_4'
outtermediate_layer_model = Model(inputs=model.input,outputs=model.get_layer(layer_name).output)
outtermediate_output = outtermediate_layer_model.predict(train_images[:1])
arr = np.loadtxt("conv5.txt", delimiter=",", dtype=int).reshape(outtermediate_output.shape)
diff = np.abs(arr- outtermediate_output)
print("Layer 5 Conv", np.sum(diff))

layer_name = 'quant_conv2d_5'
outtermediate_layer_model = Model(inputs=model.input,outputs=model.get_layer(layer_name).output)
outtermediate_output = outtermediate_layer_model.predict(train_images[:1])
arr = np.loadtxt("conv6.txt", delimiter=",", dtype=int).reshape(outtermediate_output.shape)
diff = np.abs(arr- outtermediate_output)
print("Layer 6 Conv", np.sum(diff))

layer_name = 'quant_dense'
outtermediate_layer_model = Model(inputs=model.input,outputs=model.get_layer(layer_name).output)
outtermediate_output = outtermediate_layer_model.predict(train_images[:1])
arr = np.loadtxt("conv7.txt", delimiter=",", dtype=int).reshape(outtermediate_output.shape)
diff = np.abs(arr- outtermediate_output)
print("Layer 7 Conv", np.sum(diff))

layer_name = 'quant_dense_1'
outtermediate_layer_model = Model(inputs=model.input,outputs=model.get_layer(layer_name).output)
outtermediate_output = outtermediate_layer_model.predict(train_images[:1])
arr = np.loadtxt("conv8.txt", delimiter=",", dtype=int).reshape(outtermediate_output.shape)
diff = np.abs(arr- outtermediate_output)
print("Layer 8 Conv", np.sum(diff))

layer_name = 'quant_dense_2'
outtermediate_layer_model = Model(inputs=model.input,outputs=model.get_layer(layer_name).output)
outtermediate_output = outtermediate_layer_model.predict(train_images[:1])
arr = np.loadtxt("conv9.txt", delimiter=",", dtype=int).reshape(outtermediate_output.shape)
diff = np.abs(arr- outtermediate_output)
print("Layer 9 Conv", np.sum(diff))

2023-03-25 10:05:52.666086: W tensorflow/core/util/tensor_slice_reader.cc:96] Could not open halfsame_best: FAILED_PRECONDITION: halfsame_best; Is a directory: perhaps your file is in a different file format and you need to use a different restore operator?


1/1 [==============================] - 0s 91ms/step
Layer 1 Conv 0.0
1/1 [==============================] - 0s 64ms/step
Layer 1 Sign 0.0
1/1 [==============================] - 0s 79ms/step
Layer 2 Conv 0.0
1/1 [==============================] - 0s 85ms/step
Layer 2 Sign 0.0
1/1 [==============================] - 0s 96ms/step
Layer 3 Conv 0.0
1/1 [==============================] - 0s 123ms/step
Layer 4 Conv 0.0
1/1 [==============================] - 0s 137ms/step
Layer 5 Conv 0.0
1/1 [==============================] - 0s 162ms/step
Layer 6 Conv 0.0
1/1 [==============================] - 0s 182ms/step
Layer 7 Conv 0.0
1/1 [==============================] - 0s 210ms/step
Layer 8 Conv 0.0
1/1 [==============================] - 0s 228ms/step
Layer 9 Conv 0.0
